<a href="https://colab.research.google.com/github/dimnorin/jupyter/blob/main/Backtrader_Genetic_Optimize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
% pip install backtrader

     |████████████████████████████████| 419kB 4.1MB/s 


In [2]:
% pip install gradient-free-optimizers

     |████████████████████████████████| 71kB 3.3MB/s 


In [3]:
% pip install yfinance

     |████████████████████████████████| 6.3MB 5.5MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.60-py2.py3-none-any.whl size=23819 sha256=39b5857f17a96675525bbb20435fad2fd2a6559ed5be4fdd02a399371648e871
  Stored in directory: /root/.cache/pip/wheels/f0/be/a4/846f02c5985562250917b0ab7b33fff737c8e6e8cd5209aa3b
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [4]:
import time
import numpy as np
import pandas as pd
import backtrader as bt
import pandas_datareader.data as web
import yfinance as yf

from gradient_free_optimizers import *
from datetime import datetime

In [5]:
class SmaCross(bt.SignalStrategy):
    params = (
        ('fast', 10),
        ('slow', 30),
    )
    def __init__(self):
        sma1, sma2 = bt.ind.SMA(period=self.p.fast), bt.ind.SMA(period=self.p.slow)
        crossover = bt.ind.CrossOver(sma1, sma2)
        self.signal_add(bt.SIGNAL_LONG, crossover)

In [6]:
data = bt.feeds.PandasData(dataname=yf.download('TSLA', '2018-01-01', '2019-01-01'))

[*********************100%***********************]  1 of 1 completed


# Gradient Optimizers

In [11]:
def runstrat(p):
    cerebro = bt.Cerebro()
    cerebro.adddata(data)
    cerebro.addstrategy(SmaCross, fast = p["fast"], slow = p["slow"])
    cerebro.run()

    return cerebro.broker.getvalue()

In [12]:
def run_optimizer(class_name):
    start = time.time()

    search_space = {
        "fast": np.arange(5, 150, 2),
        "slow": np.arange(50, 150, 2),
    }

    iterations = 200

    module = __import__('gradient_free_optimizers')
    class_ = getattr(module, class_name)
    opt = class_(search_space)
    opt.search(runstrat, n_iter=iterations, verbosity=[])
    end = time.time()
    print(f'{class_name}: score:{opt.best_score}, time:{"%.2f" % (end-start)} s, para:{opt.best_para}')

In [13]:
optimizers = ['HillClimbingOptimizer',
'RepulsingHillClimbingOptimizer',
'SimulatedAnnealingOptimizer',
'RandomSearchOptimizer',
'RandomRestartHillClimbingOptimizer',
'RandomAnnealingOptimizer',
'ParallelTemperingOptimizer',
'ParticleSwarmOptimizer',
'EvolutionStrategyOptimizer',
'DecisionTreeOptimizer']

for o in optimizers:
  run_optimizer(o)

HillClimbingOptimizer: score:10023.95000076294, time:7.72 s, para:{'fast': 129, 'slow': 54}
RepulsingHillClimbingOptimizer: score:10023.95000076294, time:15.99 s, para:{'fast': 119, 'slow': 60}
SimulatedAnnealingOptimizer: score:10023.95000076294, time:7.24 s, para:{'fast': 131, 'slow': 52}
RandomSearchOptimizer: score:10026.352001190186, time:20.60 s, para:{'fast': 61, 'slow': 56}
RandomRestartHillClimbingOptimizer: score:10023.95000076294, time:10.14 s, para:{'fast': 127, 'slow': 52}
RandomAnnealingOptimizer: score:10023.95000076294, time:8.63 s, para:{'fast': 125, 'slow': 56}
ParallelTemperingOptimizer: score:10021.9880027771, time:15.08 s, para:{'fast': 147, 'slow': 50}
ParticleSwarmOptimizer: score:10030.186000823975, time:15.71 s, para:{'fast': 61, 'slow': 54}
EvolutionStrategyOptimizer: score:10023.95000076294, time:14.98 s, para:{'fast': 131, 'slow': 52}
DecisionTreeOptimizer: score:10035.94200515747, time:5.05 s, para:{'fast': 57, 'slow': 56}


# Build-in Optimize

In [9]:
start = time.time()
cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.optstrategy(SmaCross, fast = [x for x in range(5, 150, 2)], slow = [x for x in range(50, 150, 2)])
results = cerebro.run(optreturn=False)

sorted_results = sorted(results, key=lambda x: x[0].broker.cash, reverse=True)
end = time.time()

In [10]:
print(f'Build-in brutforce: score:{sorted_results[0][0].broker.cash}, time:{"%.2f" % (end-start)} s, para: "fast":{sorted_results[0][0].p.fast}, "slow":{sorted_results[0][0].p.slow}')

Build-in brutforce: score:10035.94200515747, time:207.48 s, para: "fast":57, "slow":56
